In [0]:
!pip install tensorflow==1.13.2
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

     |████████████████████████████████| 92.6MB 65kB/s 
     |████████████████████████████████| 368kB 41.6MB/s 
     |████████████████████████████████| 3.2MB 37.5MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [1]:
!pip install utils
import utils
!pip install core
import core as flags_core


ERROR: Could not find a version that satisfies the requirement core (from versions: none)
ERROR: No matching distribution found for core


ModuleNotFoundError: ignored

In [0]:
import sys

sys.path.append("/content/models")
!export PYTHONPATH=$PYTHONPATH:/content/models
import tensorflow as tf
from absl import app as absl_app
from absl import flags
#import data.dataset as dataset
#import decoder
#import speechrecogModel
from flags import core as flags_core
from official.utils.logs import hooks_helper
from official.utils.logs import logger
from official.utils.misc import distribution_utils
from official.utils.misc import model_helpers


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

ModuleNotFoundError: ignored

In [0]:
VOCAB_FILE = os.path.join(os.path.dirname(__file__), "data/vocabulary.txt")


NameError: ignored

In [0]:
def ctc_loss(label_len, ctc_X_length, labels, logits):
  label_len = tf.to_int32(tf.squeeze(label_len))
  ctc_X_length = tf.to_int32(tf.squeeze(ctc_X_length))
  sparse_labels = tf.to_int32(tf.keras.backend.ctc_label_dense_to_sparse(labels, label_len))
  y_pred = tf.log(tf.transpose(logits, perm=[1, 0, 2]) + tf.keras.backend.epsilon())
  return tf.expand_dims(tf.nn.ctc_loss(labels=sparse_labels, inputs=y_pred,sequence_length=ctc_X_length),axis=1)

In [0]:
def evaluate_model(estimator,speech_labels,entries,X_fn_eval):
  predictions=estimator.predict(input_fn=X_fn_eval)
  probabilities=[pred["probabilities"] for pred in predictions]
  num_examples=len(probs)
  targets=[entry[2] for entry in entries]
  total_wer=0
  greedy_decoder=decoder.DeepSpeechDecoder(speech_labels)
  for i in range(num_examples):
    decoded_string=greedy_decoder.decode(probs[i])
    total_wer+=greedy_decoder.wer(decoded_string,targets[i])/float(len(targets[i].split()))
  total_wer/=num_examples
  global_step = estimator.get_variable_value(tf.GraphKeys.GLOBAL_STEP)
  results={"WER":total_wer,tf.GraphKeys.GLOBAL_STEP:Global_step}
  return results

In [0]:
def length_after_conv(max_time_steps,ctc_time_steps,X_length):
  ctc_X_length = tf.to_float(tf.multiply(X_length, ctc_time_steps))
  return tf.to_int32(tf.floordiv(ctc_X_length, tf.to_float(max_time_steps)))

In [0]:
def Model(features,labels,mode,params):
  num_classes=params["num_classes"]
  X_length=features["X_length"]
  label_length=features["label_length"]
  features=features["features"]
  model=speechrecogModel.DeepSpeechModel(flag_obj.rnn_layers_count,flags_obj.rnn_type,flags_obj.is_Bidirectional,flags_obj.rnn_hidden_size,num_classes,flags_obj.bias)
  if(mode==tf.estimator.ModeKeys.PREDICT):
    logits=model(features,training=False)
    predictions={
        "classes":tf.argmax(logits,axis=2),
        "probabilities":tf.nn.softmax(logits),
        "logits":logits
    }
    return tf.estimator.EstimatorSpec(mode,predications)
  logits=model(features,training=True)
  probabilities=tf.nn.softmax(logits)
  ctc_X_length=length_after_conv(tf.shape(features)[1],tf.shape(probabilities)[1],X_length)
  loss=tf.reduce_mean(ctc_loss(label_length,ctc_X_length,labels,probabilities))
  optimizer=tf.train.AdamOptimizer(learning_rate=flags_obj.learning_rate)
  Global_step=tf.train.get_or_create_global_step()
  minimize=optimizer.minimize(loss,Global_step)
  update=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  train=tf.group(minimize,update)
  return tf.estimator.EstimatorSpec(mode,loss,train)
  

In [0]:
def generate_dataset(data_dir):
  audio_conf = dataset.AudioConfig(sample_rate=flags_obj.sample_rate,window_ms=flags_obj.window_ms,stride_ms=flags_obj.stride_ms,normalize=True)
  training_data_conf = dataset.DatasetConfig(audio_conf,data_dir,flags_obj.vocabulary_file,flags_obj.sortagrad)
  speech_data = dataset.DeepSpeechDataset(training_data_conf)
  return speech_data

In [0]:
def run_deep_speech(_):
  tf.set_random_seed(flags_obj.seed)
  tf.logging.info("Data preprocessing...")
  train_speech_dataset = generate_dataset(flags_obj.train_data_dir)
  eval_speech_dataset = generate_dataset(flags_obj.eval_data_dir)
  num_classes = len(train_speech_dataset.speech_labels)

  # Use distribution strategy for multi-gpu training
  num_gpus = flags_core.get_num_gpus(flags_obj)
  distribution_strategy = distribution_utils.get_distribution_strategy(num_gpus=num_gpus)
  run_config = tf.estimator.RunConfig(
      train_distribute=distribution_strategy)

  estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      model_dir=flags_obj.model_dir,
      config=run_config,
      params={
          "num_classes": num_classes,
      }
  )

  run_params = {
      "batch_size": flags_obj.batch_size,
      "train_epochs": flags_obj.train_epochs,
      "rnn_hidden_size": flags_obj.rnn_hidden_size,
      "rnn_layers_count": flags_obj.rnn_layers_count,
      "rnn_type": flags_obj.rnn_type,
      "is_Bidirectional": flags_obj.is_Bidirectional,
      "bias": flags_obj.bias
  }

  dataset_name = "LibriSpeech"
  benchmark_logger = logger.get_benchmark_logger()
  benchmark_logger.log_run_info("deep_speech", dataset_name, run_params,
                                test_id=flags_obj.benchmark_test_id)

  train_hooks = hooks_helper.get_train_hooks(
      flags_obj.hooks,
      model_dir=flags_obj.model_dir,
      batch_size=flags_obj.batch_size)


In [0]:
  def input_train():
    return dataset.input_fn(
        per_replica_batch_size, train_speech_dataset)

In [0]:
  def input_eval():
    return dataset.input_fn(
        per_replica_batch_size, eval_speech_dataset)

  total_training_cycle = (flags_obj.train_epochs // flags_obj.epochs_between_evals)
  for cycle_index in range(total_training_cycle):
    tf.logging.info("Starting a training cycle: %d/%d",
                    cycle_index + 1, total_training_cycle)

    train_speech_dataset.entries = dataset.batch_wise_dataset_shuffle(
        train_speech_dataset.entries, cycle_index, flags_obj.sortagrad,
        flags_obj.batch_size)

    estimator.train(input_fn=input_fn_train, hooks=train_hooks)

    tf.logging.info("Starting to evaluate...")

    eval_results = evaluate_model(
        estimator, eval_speech_dataset.speech_labels,
        eval_speech_dataset.entries, input_fn_eval)

    benchmark_logger.log_evaluation_result(eval_results)
    tf.logging.info(
        "Iteration {}: WER = {:.2f}".format(
            cycle_index + 1, eval_results["WER"]))

    if model_helpers.past_stop_threshold(
        flags_obj.wer_threshold, eval_results["WER"]):
      break

NameError: ignored

In [0]:
def define_deep_speech_flags():
 
  flags_core.define_base(
      data_dir=False,  
      export_dir=True,
      train_epochs=True,
      hooks=True,
      num_gpu=True,
      epochs_between_evals=True
  )
  flags_core.define_performance(
      num_parallel_calls=False,
      inter_op=False,
      intra_op=False,
      synthetic_data=False,
      max_train_steps=False,
      dtype=False
  )
  flags_core.define_benchmark()
  flags.adopt_module_key_flags(flags_core)

  flags_core.set_defaults(
      model_dir="/tmp/deep_speech_model/",
      export_dir="/tmp/deep_speech_saved_model/",
      train_epochs=10,
      batch_size=128,
      hooks="")

  # Deep speech flags
  flags.DEFINE_integer(
      name="seed", default=1,
      help=flags_core.help_wrap("The random seed."))

  flags.DEFINE_string(
      name="train_data_dir",
      default="/tmp/librispeech_data/test-clean/LibriSpeech/test-clean.csv",
      help=flags_core.help_wrap("The csv file path of train dataset."))

  flags.DEFINE_string(
      name="eval_data_dir",
      default="/tmp/librispeech_data/test-clean/LibriSpeech/test-clean.csv",
      help=flags_core.help_wrap("The csv file path of evaluation dataset."))

  flags.DEFINE_bool(
      name="sortagrad", default=True,
      help=flags_core.help_wrap(
          "If true, sort examples by audio length and perform no "
          "batch_wise shuffling for the first epoch."))

  flags.DEFINE_integer(
      name="sample_rate", default=16000,
      help=flags_core.help_wrap("The sample rate for audio."))

  flags.DEFINE_integer(
      name="window_ms", default=20,
      help=flags_core.help_wrap("The frame length for spectrogram."))

  flags.DEFINE_integer(
      name="stride_ms", default=10,
      help=flags_core.help_wrap("The frame step."))

  flags.DEFINE_string(
      name="vocabulary_file", default=_VOCABULARY_FILE,
      help=flags_core.help_wrap("The file path of vocabulary file."))

  # RNN related flags
  flags.DEFINE_integer(
      name="rnn_hidden_size", default=800,
      help=flags_core.help_wrap("The hidden size of RNNs."))

  flags.DEFINE_integer(
      name="rnn_hidden_layers", default=5,
      help=flags_core.help_wrap("The number of RNN layers."))

  flags.DEFINE_bool(
      name="use_bias", default=True,
      help=flags_core.help_wrap("Use bias in the last fully-connected layer"))

  flags.DEFINE_bool(
      name="is_bidirectional", default=True,
      help=flags_core.help_wrap("If rnn unit is bidirectional"))

  flags.DEFINE_enum(
      name="rnn_type", default="gru",
      enum_values=deep_speech_model.SUPPORTED_RNNS.keys(),
      case_sensitive=False,
      help=flags_core.help_wrap("Type of RNN cell."))

  # Training related flags
  flags.DEFINE_float(
      name="learning_rate", default=5e-4,
      help=flags_core.help_wrap("The initial learning rate."))

  # Evaluation metrics threshold
  flags.DEFINE_float(
      name="wer_threshold", default=None,
      help=flags_core.help_wrap(
          "If passed, training will stop when the evaluation metric WER is "
          "greater than or equal to wer_threshold. For libri speech dataset "
          "the desired wer_threshold is 0.23 which is the result achieved by "
          "MLPerf implementation."))


In [0]:
def main(_):
  with logger.benchmark_context(flags_obj):
    run_deep_speech(flags_obj)

In [0]:
if __name__ == "__main__":
  tf.logging.set_verbosity(tf.logging.INFO)
  define_deep_speech_flags()
  flags_obj = flags.FLAGS
  absl_app.run(main)

NameError: ignored